In [1]:
# Used packages
from preprocess import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import os
import seaborn as sns
from preprocess import preprocessing
from preparation import preparation


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Users\diego\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''
Plots the confussion matrix from our system results

Receives a vector with the system results and another one with the actual results
Does not return anything but it displays the confussion matrix
'''
def print_confussion_matrix(y_test, y_pred):
    sns.heatmap((confusion_matrix(y_test,y_pred)), annot=True, fmt="d",cmap="crest")
    plt.title("Confusion matrix")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()

'''
Builds a dictionary with the word frequency counts of a string

Receives a preprocessed text
Returns a dictionary with the word frequency counts
'''
def build_word_frequency_histogram(preprocessed_str):
    wordfreq = {}

    # If word has already been found, add 1 to its count. If not, add the word to the dictionary
    for word in preprocessed_str.split():
        if word not in wordfreq:
            wordfreq[word] = 0
        wordfreq[word] += 1
    
    # REturn 10 most frequent words
    return dict(sorted(wordfreq.items(), key=lambda item: item[1], reverse=True)[:10])

In [3]:
''' 
Creates subplots and displays all bar charts of histograms

Receives all the system results
Does not return anything but it displays all the histograms, ROC Curve and Confussion Matrix
'''
def create_subplots(maxhistograms, maxhistogramsoriginals, maxfilename, maxfilenameoriginals ,fpr, tpr, actual_results, system_results):
    # Plot Histograms
    fig, axs = plt.subplots(len(maxhistograms), 2, figsize=(20, 40))
    fig.tight_layout(pad=10)
    for i, histogram in enumerate(maxhistograms):
        categories = list(histogram.keys())[:20]
        frequencies = list(histogram.values())[:20]
        axs[i][0].bar(categories, frequencies)            
        axs[i][0].set_xlabel('Categories')
        axs[i][0].set_ylabel('Frequency')
        axs[i][0].set_title(f'Word frequency histogram for {maxfilename[i]}')
        axs[i][0].tick_params(axis='x', rotation=90)
        categories = list(maxhistogramsoriginals[i].keys())[:20]
        frequencies = list(maxhistogramsoriginals[i].values())[:20]
        axs[i][1].bar(categories, frequencies)
        axs[i][1].set_xlabel('Categories')
        axs[i][1].set_ylabel('Frequency')
        axs[i][1].set_title(f'Word frequency histogram for {maxfilenameoriginals[i]}')
        axs[i][1].tick_params(axis='x', rotation=90)

    # Plot ROC Curve
    plt.figure(figsize=(5, 5))
    roc_auc = metrics.auc(fpr, tpr)
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.title('ROC curve')
    plt.xlabel("Error")
    plt.ylabel("True Positive Rate")
    plt.show()
    
    # Plot Confussion Matrix
    print_confussion_matrix(actual_results, system_results)

In [4]:
# Stores all the plagiarism results for every comparison
dictPlag = {}

# Actual results of our tests. 1 = Plagiarism, 0 = Original
actual_results = [1,1,0,0,1,1,0,0,0,0,0,0,0,0,1]

# Results found by our system when doing the plagiarism analysis
system_results = []

'''
Main decision function that handles all the main logic in our system.
First, it gathers all the original and suspicious documents for preprocessing.
Then, it calls for the calculation of the cosine similarity between a suspicious and an original text.
Afterwards, it determines whether there is plagiarism or not and it calculates the word frequency histograms.
Finally, it displays all the histograms, ROC Curve and Confussion Matrix.

Receives a file path to the original and suspicious files
Does not return anything but it concludes by showing all the result graphs.
'''
def decision(file_path_originals, file_path_suspicious):
   
    # Preprocessing original texts
    original_texts = [file for file in os.listdir(file_path_originals) if os.path.isfile(os.path.join(file_path_originals, file))]
    processed_original_texts = []
    for original_text in original_texts:
        processed_original_texts.append(preprocessing(file_path_originals + "/" + original_text))

    print("Starting plagiarism detection...")
    print("\n")
    # Preprocessing suspicious texts
    suspicious_texts = [file for file in os.listdir(file_path_suspicious) if os.path.isfile(os.path.join(file_path_suspicious, file))]
    processed_suspicious_texts = []
    for suspicious_text in suspicious_texts:
        processed_suspicious_texts.append(preprocessing(file_path_suspicious + "/" + suspicious_text))
       
    # Creating histograms for suspicious texts
    maxplagiarism = [0 for _ in range(len(processed_suspicious_texts))]
    maxhistograms = [{} for _ in range(len(processed_suspicious_texts))]
    maxhistogramsoriginal = [{} for _ in range(len(processed_suspicious_texts))]
    maxfilename = ["" for _ in range(len(processed_suspicious_texts))]
    maxfilenameoriginal = ["" for _ in range(len(processed_suspicious_texts))]
    
    # Comparing suspicious text with original texts
    for k, processed_suspicious_text in enumerate(processed_suspicious_texts):
        print("Analyzing Suspicious text: ", suspicious_texts[k])
        plagiarized_check = False

        for i, processed_original_text in enumerate(processed_original_texts):
            result = preparation(processed_suspicious_text, processed_original_text)
            print(result)
            # Check if plagiarism is detected
            if result > 0.7:
                print("\tPlagiarism detected in file: ", original_texts[i])
                print(f'\tMaximum percentage of plagiarism: {result*100:.1f}%')
                print("\t⚠️ ⚠️ ⚠️ ⚠️ ⚠️ ⚠️\n")
                
                if dictPlag.get(suspicious_text) is None:
                    dictPlag[suspicious_text] = [[original_texts[i],result]]
                else:
                    dictPlag[suspicious_text].append([original_texts[i], result])
                    
                # When plagiarism is detected, append 1 to system results
                if not plagiarized_check:
                    system_results.append(1)
                    plagiarized_check = True
                
                # Check if plagiarism is the highest detected
                if maxplagiarism[k] <= result:
                    maxplagiarism[k] = result
                    maxfilename[k] = suspicious_texts[k]
                    maxfilenameoriginal[k] = original_texts[i]
                    maxhistograms[k] = build_word_frequency_histogram(processed_suspicious_text)
                    maxhistogramsoriginal[k] = build_word_frequency_histogram(processed_original_text)
                
                    
        # When there is no plagiarism, append 0 to system results
        if not plagiarized_check:
            print("\tNo plagiarism detected\n")
            system_results.append(0)


    # Remove empty histograms
    maxhistograms = [histogram for histogram in maxhistograms if histogram]
    maxfilename = [filename for filename in maxfilename if filename]
    maxfilenameoriginal = [filename for filename in maxfilenameoriginal if filename]
    maxhistogramsoriginal = [histogram for histogram in maxhistogramsoriginal if histogram]

    # Print results
    tn, fp, fn, tp = confusion_matrix(actual_results, system_results).ravel()
    print(f'Predicted Results: {system_results}')
    print(f'Actual Results: {actual_results}')
    fpr, tpr, thresholds = metrics.roc_curve(actual_results, system_results, pos_label=1)
    print(f'True Positive: {tp}')
    print(f'False Positive: {fp}')
    print(f'True Negative: {tn}')
    print(f'False Negative: {fn}')
    print("False Positive Rate: ", fp/(fp+tn))
    print("True Positive Rate: ", tp/(tp+fn))
    print("AUC:", metrics.auc(fpr, tpr))

    # Display graphs
    # create_subplots(maxhistograms, maxhistogramsoriginal ,maxfilename, maxfilenameoriginal ,fpr, tpr, actual_results, system_results)
    


In [5]:
# Call the main system function to kick-off the plagiarism detector
decision("original_files", "suspicious_files")


Starting plagiarism detection...


Analyzing Suspicious text:  FID-01.txt
torch.Size([1, 1])
tensor(0.4963)
torch.Size([1, 1])
tensor(0.2232)
torch.Size([1, 1])
tensor(0.0974)
torch.Size([1, 1])
tensor(0.1947)
torch.Size([1, 1])
tensor(0.4577)
torch.Size([1, 1])
tensor(0.4564)
torch.Size([1, 1])
tensor(0.2773)
torch.Size([1, 1])
tensor(0.2431)
torch.Size([1, 1])
tensor(0.5059)
torch.Size([1, 1])
tensor(0.7433)
	Plagiarism detected in file:  org-010.txt
	Maximum percentage of plagiarism: 74.3%
	⚠️ ⚠️ ⚠️ ⚠️ ⚠️ ⚠️

torch.Size([1, 1])
tensor(0.0956)
torch.Size([1, 1])
tensor(0.2348)
torch.Size([1, 1])
tensor(0.1916)
torch.Size([1, 1])
tensor(0.1477)
torch.Size([1, 1])
tensor(0.1460)
torch.Size([1, 1])
tensor(0.2975)
torch.Size([1, 1])
tensor(0.1291)
torch.Size([1, 1])
tensor(0.2499)
torch.Size([1, 1])
tensor(0.1264)
torch.Size([1, 1])
tensor(0.2741)
torch.Size([1, 1])
tensor(0.2241)
torch.Size([1, 1])
tensor(0.2834)
torch.Size([1, 1])
tensor(0.2004)
torch.Size([1, 1])
tensor(0.1966)
torch.